# Gemini Trial   <font size="2">by Enos</font>
<b>Focus</b>
<ol>
    <li><font color="blue">車牌辨識</font></li>
    <li><font color="blue">聊天</font></li>
    <li><font color="blue">聊天帶入 Function</font></li>
</ol>

<b>Role</b>
<ol>
    <li>Vertex AI 使用者</li>
</ol>

<b>Dependences</b>
<ol>
    <li>google-cloud-aiplatform</li>
</ol>

In [ ]:
!pip install --upgrade google-cloud-aiplatform

## 車牌辨識
Vertext AI/ VERTEX AI STUDIO: 多模態

In [ ]:
import os
from io import BytesIO
from time import time

import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from PIL import Image

# gcloud auth application-default login
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'YOUR_SERVICE'

start = time()

def generate(data, prompt):
    model = GenerativeModel("gemini-pro-vision")
    response = model.generate_content(
        [data, prompt],
        generation_config = {
            "max_output_tokens": 100,
            "temperature": 0,
            "top_k": 1,  # 限制候選 tokens 為機率最高的 top_k 個
            "top_p": 0.75  # 限制候選 tokens 為加總機率 (從機率機率開始) 達到 top_p 的 tokens
        },
    )
    return response

# calling gemini
prompt = "車牌號碼為何 ? 請遵循下列規則: 1. 車牌號碼中間一律有減號 2. 只提供車牌號碼，不須說明"
your_filename = 'some_plate.jpg'
image = Image.open(your_filename)
image_bytes = BytesIO()
image.save(image_bytes, format='JPEG')
data = image_bytes.getvalue()
#with open(your_filename, 'rb') as f:
#    data = f.read()
imagedata = Part.from_data(data=data, mime_type='image/jpeg')
r = generate(imagedata, prompt)

# output result
plate = r.text.strip()
usage = r._raw_response.usage_metadata
elapsed = time()-start
print(f"{plate}\n{usage}{elapsed:.1f} secs")

## 聊天
ref: https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/send-chat-prompts-gemini
<table>
    <ol>
        <li><font color="blue">SDK 能夠自行 cache history</font></li>
        <li><font color="red">SDK 移除 system prompt/ context 的功能，須硬塞一組 system prompt 至第一筆 history 或於第一筆聊天內容前加上 system prompt</font></li>
    </ol>
</table>

In [ ]:
import os

import vertexai
from vertexai.preview.generative_models import (
    GenerativeModel,
    Part,
    HarmCategory,
    HarmBlockThreshold,
    Content
)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'YOUR_SERVICE'

a = Content(role="user", parts=[Part.from_text('你是誰')])
b = Content(role="model", parts=[Part.from_text("我是長輩聊天機器人，專責與長輩以繁體中文閒聊，充滿愛心、耐心，與包容")])
chat_his = [a, b]

config = {
    "max_output_tokens": 2048,
    "temperature": 0.9,
    "top_p": 1
}

safety_config = {
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
    }

model = GenerativeModel("gemini-pro")
chat = model.start_chat(history=chat_his)
r1 = chat.send_message("天氣好，有何建議", generation_config=config, safety_settings=safety_config)
print(r1.text)
print(chat.history)
print("-----")
r2 = chat.send_message("哪種便宜", generation_config=config, safety_settings=safety_config)
print(r2.text)
print(chat.history)
print("-----")
r3 = chat.send_message("適合晚上的", generation_config=config, safety_settings=safety_config)
print(r3.text)
print(chat.history)

## 聊天帶入 Function
ref 1: https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling <br>
ref 2: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb

In [ ]:
import os

import vertexai
from vertexai.preview.generative_models import (
    GenerativeModel,
    FunctionDeclaration,
    Tool
)
from wea import WeaG

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'YOUR_SERVICE'

w = WeaG()
model = GenerativeModel("gemini-pro")

def query(message):
    weag = FunctionDeclaration(
        name="get_current_weather",
        description="Get the current weather in a given location",
        parameters={
            "type": "object",
            "properties": {"location": {"type": "string", "description": "Location"}},
        },
    )

    weather_tool = Tool(
        function_declarations=[weag],
    )

    response = model.generate_content(
        message,
        generation_config={"temperature": 0},
        tools=[weather_tool],
    )
    
    a = response.candidates[0].content.parts[0].function_call
    if a and a.name == 'get_current_weather':
        return a.args.get('location')
    return None

message = '中和天氣如何 ?'
location = query(message)
if location:
    r = w.grab(location)
    print(f'{location}\n時間: {r["O"]}\n溫度: {r["T"]:.1f}°C\n濕度: {r["H"]:.0%}\n雨量: {r["R"]:.0f} mm')